#### Install python dependencies

In [ ]:
%pip install jsonpickle dacite shapely

#### Run the planning server (should be executed only once)

In [ ]:
import py_planning
py_planning.init()

def display_simulator(case_name):
    from IPython.display import IFrame, display
    url = 'http://127.0.0.1:8008?scenario=' + case_name
    print("You can also open in browser: " + url)
    display(IFrame(url, width="100%", height=650))

### Simulation

#### Place your planning code here, then run the cells below and watch the simulation

In [ ]:
from py_planning.data_types import PlannedPath, PlannedState, State  # data types used by planner interface
from shapely.geometry import LineString, Point

"""
find closest point on a polyline to the given point
"""
def get_index_of_closest_point(line: LineString, point: Point):
    closest_point_index = None
    min_distance = float('inf')

    for i, line_point in enumerate(line.coords):
        line_point = Point(line_point)
        distance = point.distance(line_point)
        if distance < min_distance:
            min_distance = distance
            closest_point_index = i

    return closest_point_index

"""
This function is called by the simulator for each tick.
It should return recent planned trajectory up to date with the environment state.
'state' parameter contains current world observations and vehicle state.
"""
def do_plan(state: State) -> PlannedPath:
    vehicle_pose = state.vehicle_pose
    vehicle_pos = Point(vehicle_pose.pos.x, vehicle_pose.pos.y)  # current position of the AV

    centerline = LineString([(p.x, p.y) for p in state.lane_path.centerline])

    closest_index = get_index_of_closest_point(centerline, vehicle_pos)

    # we leave some previous poses to make AV control stable
    prev_poses_count = 3
    max_poses_count = 50
    first_pose_index = max(closest_index - prev_poses_count, 0)

    # as a baseline here we just follow the centerline
    planned_states = [
        PlannedState(pos=p) for p in state.lane_path.centerline
    ][first_pose_index:first_pose_index+ max_poses_count]

    return PlannedPath(states=planned_states)


#### Case 1

In [ ]:
display_simulator('one-car-overtake')

# run the case in the simulator, watch the visualization
py_planning.run_planner(
    do_plan,
    stop_on_fail=True  # set to False to continue planning after case fail (useful for debugging, can restart scenario in simulator)
)

#### Case 2

In [ ]:
display_simulator('rough-road')

# run the case in the simulator, watch the visualization
py_planning.run_planner(
    do_plan,
    stop_on_fail=True  # set to False to continue planning after case fail (useful for debugging, can restart scenario in simulator)
)